## Previo
- Agarrar los datos del tp anterior
- Bajarse el dataset de testing de Kaggle (no usado actualmente)

In [21]:
from __future__ import division
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

## Inicializo el DF

In [2]:
dtypes = {
    'country_name': str,
    'expenses': object,
    'id': str,
    'title': str,
    'state_name': str,
    'lat': float,
    'lon': float
}

cols = range(6) + range(8, 18) + [20, 21] + range(23, 27)
parse_dates = ['created_on']

df = pd.read_csv('data_gba_total.csv', dtype=dtypes, usecols=cols, parse_dates=parse_dates)
df = df.select_dtypes(['number'])
df.sample(5)

,floor,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,price_per_m2,rooms,surface_covered_in_m2,surface_in_m2,surface_total_in_m2
635885,NaN,NaN,NaN,NaN,108000.0,1724598.00,108000.00,2250.000000,2.0,48.0,NaN,NaN
1403749,NaN,3435359.0,-34.565321,-58.471603,74000.0,717851.80,74000.00,2242.424242,1.0,33.0,NaN,37.0
379561,NaN,NaN,NaN,NaN,5026201.0,5026200.98,314757.24,55846.677778,NaN,90.0,NaN,NaN
2258476,NaN,3433359.0,NaN,NaN,50000.0,431710.00,50000.00,NaN,3.0,NaN,330.0,NaN
945804,NaN,3436077.0,-34.578282,-58.467671,85000.0,585182.50,85000.00,NaN,2.0,NaN,41.0,NaN


### Cleanup

Cleanup muy malo a mejorar, hay que usar mas columnas, actualmente descarto demasiada informacion para poder tener un dataset limpio usable mas rapido

In [3]:
df = df.fillna(value=0)
df = df.loc[df['price_aprox_usd'] > 5000]
price = df['price_aprox_usd']
for column in df.columns:
    if 'price' in column:
        del df[column]
        continue


del df['surface_in_m2']  # Existen otras columnas, surface_total_in_m2
del df['geonames_id']

In [8]:
df.shape

(2085663, 6)

In [19]:
# Parto el dataframe en train y test


Busco el mejor hiperparametro, haciendo grid search. En el caso de Random Forest el unico hiper parametro es la cantidad de features (columnas) a usar por arbol, entonces itero los valores posibles (de 1 a 6) y me quedo con el que minimice el error.

In [13]:
param_grid = {'max_features': range(1, 7)}

r = RandomForestRegressor()
grid = GridSearchCV(r, param_grid)
grid.fit(x_train, y_train)
grid.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

Score del mejor parametro.

In [14]:
grid.best_score_

0.66783776197159994

Aca pruebo el mejor estimador encontrado por el grid search (el que usa los mejores hiperparametros encontrados) unas cuantas veces con sets random a ver cuanto da el error promedio

In [20]:
errors = []
estimator = grid.best_estimator_
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(df, price)
    estimator.fit(x_train, y_train)
    result = estimator.predict(x_test)
    errors.append(mean_squared_error(y_test, result))


In [22]:
#ERROR PROMEDIO
reduce(lambda x, y: x+y, errors) / len(errors)

36205641230.160675

In [27]:
pd.DataFrame(errors).describe()

,0
count,1.000000e+01
mean,3.620564e+10
std,1.885328e+09
min,3.280291e+10
25%,3.474570e+10
50%,3.649749e+10
75%,3.765098e+10
max,3.889870e+10
